# Import packages

In [1]:
import os
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from scipy.interpolate import griddata
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from matplotlib.backends.backend_pdf import PdfPages
rcParams['font.family'] = 'sans-serif'
directory   = '/g/data/fu5/eac/20year_run_Junde/jl4424/KC_GS'

# Read SST, GSST and GSST trend

In [2]:
dataset1   = sio.loadmat(directory+'/Figures/Figure1_trend.mat')
dataset2   = sio.loadmat(directory+'/Figures/Figure2_mean.mat')
dataset3   = xr.open_dataset(directory+'/AVISO/AVISO_monthly_SSH.nc')
gsst_trend = dataset1['gsst_trend'][:,:]*3650*1e5
gsst_p     = dataset1['gsst_p'][:,:]
oisst_lon  = dataset2['oisst_lon'][:,:]
oisst_lat  = dataset2['oisst_lat'][:,:]
sst_mean   = dataset2['oisst_mean'][:,:]
gsst_mean  = dataset2['gsst_mean'][:,:]*1e5
gsst_p[np.where(gsst_p > 0.05)] = np.nan
aviso_ssh  = dataset3.adt.mean(axis=0).transpose().values

# Plot the spatial distribution of SST, GSST and GSST trend

In [ ]:
##############################################################################################################
labels      = ['a','b','c','d','e','f']
levels1     = np.linspace(10,26,50)
levels2     = np.linspace(0,3,50)
levels3     = np.linspace(-1.0,1.0,50)
tick_marks1 = np.linspace(10,26,9)
tick_marks2 = np.linspace(0,3,7)
tick_marks3 = np.linspace(-1.0,1.0,5)
cmaps0      = cmaps.cmocean_thermal 
cmaps1      = cmocean.cm.matter
cmaps2      = cmocean.cm.balance
fig_ratio   = 0.9
fig         = plt.figure(figsize=(24, 32))
gs          = gridspec.GridSpec(3,2)
labelfont   = 30
padspacescale = 10
labelpadscale = 2
linefont      = 2
scale1        = 1.0
scale2        = 1.0
m_scale       = 50 
##############################################################################################################
for i in range(6):
    if i==0: 
        lon       = oisst_lon[80:202,120:182]      
        lat       = oisst_lat[80:202,120:182]      
        var_ssh   = aviso_ssh[80:202,120:182]      
        ssh_level = np.arange(1.099,1.101,10)
        var_mean  = sst_mean[80:202,120:182]            
    elif i==1:
        lon       = oisst_lon[640:762,120:182]      
        lat       = oisst_lat[640:762,120:182]  
        var_ssh   = aviso_ssh[640:762,120:182]  
        ssh_level = np.arange(0.249,0.251,10)
        var_mean  = sst_mean[640:762,120:182]         
    elif i==2:
        lon       = oisst_lon[80:202,120:182]      
        lat       = oisst_lat[80:202,120:182]    
        var_ssh   = aviso_ssh[80:202,120:182]    
        ssh_level = np.arange(1.099,1.101,10)
        var_mean  = gsst_mean[80:202,120:182]                     
    elif i==3:
        lon       = oisst_lon[640:762,120:182]     
        lat       = oisst_lat[640:762,120:182] 
        var_ssh   = aviso_ssh[640:762,120:182]      
        ssh_level = np.arange(0.249,0.251,10)
        var_mean  = gsst_mean[640:762,120:182]     
    elif i==4:  
        lon       = oisst_lon[80:202,120:182]      
        lat       = oisst_lat[80:202,120:182] 
        var_ssh   = aviso_ssh[80:202,120:182]     
        ssh_level = np.arange(1.099,1.101,10)
        var_mean  = gsst_trend[80:202,120:182]     
        var_p     = gsst_p[80:202,120:182] 
    else:
        lon       = oisst_lon[640:762,120:182]      
        lat       = oisst_lat[640:762,120:182]     
        var_ssh   = aviso_ssh[640:762,120:182]    
        ssh_level = np.arange(0.249,0.251,10)
        var_mean  = gsst_trend[640:762,120:182]   
        var_p     = gsst_p[640:762,120:182]    
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    ax = fig.add_subplot(gs[i])
    l, b, w, h = ax.get_position().bounds
    m   = Basemap(projection='merc',suppress_ticks=True,llcrnrlat=np.nanmin(lat),urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='c')        
    m.drawparallels(np.arange(15,50, 5),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')    
    m.drawmeridians(np.arange(0,360, 5),labels=[0,0,0,1],linewidth=linefont,dashes=[2,2],color='.5',fontsize=labelfont,family='sans-serif')
    m.drawcoastlines(color='0.1',  linewidth=linefont)
    m.fillcontinents(color='0.9', lake_color='white')
    x, y = m(lon, lat)
    yy = np.arange(1, y.shape[0], 4)
    xx = np.arange(1, x.shape[1], 4)
    if i>=4:
        points      = np.meshgrid(yy, xx)
        point_index = tuple(points)
        point_x     = x[point_index]
        point_y     = y[point_index]
        point_z     = var_p[point_index]
        point_x     = np.ravel(point_x)
        point_y     = np.ravel(point_y)
        point_z     = np.ravel(point_z)
        point_x     = point_x[~np.isnan(point_z)]
        point_y     = point_y[~np.isnan(point_z)]
    
    if i==0:
        cx,cy = m(136.5, 46.0)
        CB1 = m.contourf(x, y, var_mean,cmap=cmaps0,levels=levels1,origin='lower',extend='both')
        plt.ylabel('Latitude',fontsize=labelfont,labelpad=10*padspacescale,family='sans-serif')
    elif i==1:
        cx,cy  = m(276.5, 46.0)
        CB1 = m.contourf(x, y, var_mean,cmap=cmaps0,levels=levels1,origin='lower',extend='both')
    elif i==2:
        b=b+0.04
        cx,cy = m(136.5, 46.0)
        CB2 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels2,origin='lower',extend='both')
        plt.ylabel('Latitude',fontsize=labelfont,labelpad=10*padspacescale,family='sans-serif')
    elif i==3:
        b=b+0.04
        cx,cy  = m(276.5, 46.0)
        CB2 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels2,origin='lower',extend='both')
    elif i==4:
        b=b+0.08
        cx,cy = m(136.5, 46.0)
        CB3 = m.contourf(x, y, var_mean,cmap=cmaps2,levels=levels3,origin='lower',extend='both')
        plt.ylabel('Latitude',fontsize=labelfont,labelpad=10*padspacescale,family='sans-serif')
        plt.xlabel('Longitude',fontsize=labelfont,labelpad=5*padspacescale,family='sans-serif')
        CS1 = m.scatter(point_x,point_y,s=m_scale,c='dimgray',marker='.') 
    else:
        b=b+0.08
        cx,cy  = m(276.5, 46.0)
        CB3 = m.contourf(x, y, var_mean,cmap=cmaps2,levels=levels3,origin='lower',extend='both')
        plt.xlabel('Longitude',fontsize=labelfont,labelpad=5*padspacescale,family='sans-serif')
        CS1 = m.scatter(point_x,point_y,s=m_scale,c='dimgray',marker='.') 

    m.plot(x[0,:],  y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0],  y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    ax.spines['bottom'].set_linewidth(labelpadscale)
    plt.tick_params(axis='x',which='major',bottom='on',left='on',top='on',right='on',\
                length=25,width=2,colors='black',direction='out')
    plt.text(cx, cy,labels[i],color='xkcd:black', fontsize=1.0*labelfont, weight='bold',family='sans-serif')
    if np.mod(i,2)==0:
        ax.set_position([l, b, scale1*w, scale1*h])   
        CS2 = m.contour(x, y, var_ssh,ssh_level,linewidths=5*linefont,linestyles='solid',colors='tab:gray')
    elif np.mod(i,2)==1:
        ax.set_position([l+0.013, b, scale2*w, scale2*h])  
        CS2 = m.contour(x, y, var_ssh,ssh_level,linewidths=5*linefont,linestyles='solid',colors='tab:blue')
    m.fillcontinents(color='0.9', lake_color='white')        
cbaxes1     = fig.add_axes([0.92, 0.685, 0.01, 0.16])
cb1 = plt.colorbar(CB1,orientation='vertical',cax = cbaxes1)
cb1.set_ticks(tick_marks1)
cb1.ax.tick_params(labelsize=0.8*labelfont)
cb1.set_label(r'$\overline{SST}$ ($^\circ$C)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
cbaxes2     = fig.add_axes([0.92, 0.455, 0.01, 0.16])
cb2 = plt.colorbar(CB2,orientation='vertical',cax = cbaxes2)
cb2.set_ticks(tick_marks2)
cb2.ax.tick_params(labelsize=0.8*labelfont)
cb2.set_label(r'$\overline{|\nabla \mathrm{SST}|}$ (10$^{-5}$ $^{\circ}$C per m)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
cbaxes3     = fig.add_axes([0.92, 0.225, 0.01, 0.16])
cb3 = plt.colorbar(CB3,orientation='vertical',cax = cbaxes3)
cb3.set_ticks(tick_marks3)
cb3.ax.tick_params(labelsize=0.8*labelfont)
cb3.set_label(r'$\overline{|\nabla \mathrm{SST}|}$ trend' '\n' '(10$^{-5}$ $^{\circ}$C per m per decade)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
fig.savefig(directory+'/Figures/FigureS2_sst_gsst.png',dpi=300,bbox_inches = 'tight')